In [ ]:
import os
import pandas as pd
import time
from datetime import datetime, timedelta
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import Select
from urllib.parse import quote
from sqlalchemy import create_engine

def load_environment_variables(file_path):
    with open(file_path, 'r') as file:
        for line in file:
            key, value = line.strip().split('=', 1)
            os.environ[key] = value

if __name__ == "__main__":
    load_environment_variables('env_vars.txt')

host= os.getenv('HOST')
database=os.getenv('DATABASE')
port= os.getenv('PORT')
user=os.getenv('USER')
password=os.getenv('PASSWORD')
email=os.getenv('EMAIL')
password_sige=os.getenv('SIGE_PASSWORD')


def create_db_engine():
    user = os.getenv('USER')
    password = quote(os.getenv('PASSWORD'))
    host = os.getenv('HOST')
    port = os.getenv('PORT')
    database = os.getenv('DATABASE')
    engine = create_engine(f'postgresql://{user}:{password}@{host}:{port}/{database}')
    return engine


def consultar_db(engine, sql):
    with engine.connect() as connection:
        result = pd.read_sql(sql, connection)
    return result



In [ ]:
def esperar_input_ter_valor(driver, xpath, valor_esperado, timeout=240):
    """Espera até que o input tenha o valor desejado."""
    end_time = time.time() + timeout
    while time.time() < end_time:
        try:
            elemento = WebDriverWait(driver, 10).until(
                EC.visibility_of_element_located((By.XPATH, xpath))
            )
            elemento.clear()
            elemento.send_keys(valor_esperado)
            time.sleep(10)  # Pequena pausa para garantir que o valor seja atualizado
            if elemento.get_attribute('value') == valor_esperado:
                return True
        except Exception as e:
            print(f"Erro ao definir o valor do input: {e}")
            time.sleep(1)  # Espera antes de tentar novamente
    return False

def fechar_popup_se_presente(driver, xpath):
    """Fecha o popup se estiver presente."""
    try:
        popup_elements = driver.find_elements(By.XPATH, xpath)
        for popup_element in popup_elements:
            if popup_element.is_displayed():
                popup_element.click()
                break
    except Exception as e:
        print(f"Erro ao fechar o popup: {e}")

def realizar_login():
    service = Service(ChromeDriverManager().install())
    download_folder = "XML_SIGE"

    # Configurações do navegador
    chrome_options = Options()
    prefs = {
        "download.default_directory": download_folder,  # Define o diretório padrão de downloads
        "download.prompt_for_download": False,          # Desativa a caixa de diálogo de download
        "download.directory_upgrade": True,             # Ativa a gestão de múltiplos downloads
        #"safebrowsing.enabled": True                    # Ativa a proteção de navegação
    }
    chrome_options.add_experimental_option("prefs", prefs)
    nav = webdriver.Chrome(service=service)
    nav.get('https://app.sigecloud.com.br/login')
    nav.find_element(By.XPATH, '/html/body/form/div[3]/div/div[2]/div[1]/input').send_keys(email)
    nav.find_element(By.XPATH,'/html/body/form/div[3]/div/div[2]/div[2]/input').send_keys(password_sige)
    nav.find_element(By.XPATH, '/html/body/form/div[3]/div/div[2]/input').click()
    return nav

def selecionar_intervalo_datas_e_download(nav, data_inicio):
    nav.get('https://app.sigecloud.com.br/NFe/DownloadXML?tipodoc=XMLNFe')

    # Espera até que o select esteja presente
    select_element = WebDriverWait(nav, 10).until(
        EC.presence_of_element_located((By.XPATH, '/html/body/form/div[4]/div[2]/div/div[3]/div[1]/div[1]/ul/li[3]/select'))
    )
    select = Select(select_element)
    select.select_by_visible_text("XML NFCe")
    print("A opção 'XML NFCe' está corretamente selecionada.")

    # Configura o dropdown para "Personalizado"
    dropdown_xpath = '/html/body/form/div[4]/div[2]/div/div[3]/div[1]/div[1]/ul/li[4]/select'
    select_element = WebDriverWait(nav, 10).until(
        EC.element_to_be_clickable((By.XPATH, dropdown_xpath))
    )
    select = Select(select_element)
    select.select_by_visible_text("Personalizado")

    data_fim = data_inicio + timedelta(days=14)
    data_inicio_str = data_inicio.strftime("%d/%m/%Y")
    data_fim_str = data_fim.strftime("%d/%m/%Y")

    input_data_inicio_xpath = '/html/body/form/div[4]/div[2]/div/div[3]/div[1]/div[1]/ul/li[5]/input'
    input_data_fim_xpath = '/html/body/form/div[4]/div[2]/div/div[3]/div[1]/div[1]/ul/li[6]/input'
    time.sleep(5)  # Small pause to ensure inputs are interactable
    if esperar_input_ter_valor(nav, input_data_inicio_xpath, data_inicio_str) and esperar_input_ter_valor(nav, input_data_fim_xpath, data_fim_str):
        nav.find_element(By.XPATH, '/html/body/form/div[4]/div[2]/div/div[3]/div[1]/div[1]/ul/li[7]/button').click()

        # Aguarda até que a barra de download esteja completa
        download_bar_xpath = '/html/body/form/div[4]/div[2]/div/div[3]/div[1]/div[2]/div/div[3]/div'
        WebDriverWait(nav, 300).until(  # Espera até 5 minutos para o download completar
            EC.invisibility_of_element_located((By.XPATH, download_bar_xpath))
        )

        time.sleep(3)  # Espera adicional de 3 segundos após o download completar
        print(f'Data {data_inicio_str} a {data_fim_str} processada com sucesso.')
        return [data_inicio_str, data_fim_str]
    else:
        print("Não foi possível definir as datas nos inputs corretamente.")
        return None

        
def salvar_datas_csv(datas, filename='datas_processadas.csv'):
    df = pd.DataFrame(datas, columns=['Data Inicio', 'Data Fim'])
    df.to_csv(filename, mode='a', header=not os.path.exists(filename), index=False)

if __name__ == "__main__":
    load_environment_variables('env_vars.txt')
    engine = create_db_engine()
    try:
        nav = realizar_login()
        datas_processadas = []
        ultima_data_tabela = consultar_db(engine, ''' SELECT MAX("dhEmi") AS max_dhemi FROM sige_notafiscal_xml ''')
        data_inicio = ultima_data_tabela.at[0, 'max_dhemi'] - timedelta(days=90)
        

        while data_inicio < datetime.now():
            data_fim = data_inicio + timedelta(days=14)
            if data_fim > datetime.now():
                data_fim = datetime.now()

            resultado = selecionar_intervalo_datas_e_download(nav, data_inicio)
            if resultado:
                datas_processadas.append(resultado)
                data_inicio += timedelta(days=15)
            else:
                print(f"Falha ao processar o intervalo iniciado em {data_inicio.strftime('%d/%m/%Y')}.")
                break

        salvar_datas_csv(datas_processadas)
        print("Processo concluído com sucesso!")
    except Exception as e:
        print(f"Ocorreu um erro: {e}")
